In [1]:
# Michael Spearing
# Started: April 26, 2017
# Data Science Lab

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import pygame
import time
pygame.mixer.init()
pygame.mixer.music.load("beep-07.wav")


In [3]:
def alert(num):
    for x in range(num):
        pygame.mixer.music.play()
        time.sleep(2)

In [4]:
file = "items-2016-01-01-2017-01-01-SQEXPORT.csv"
data_Full = pd.read_csv(file)
data_Full.head()
features = ["Date", "Time", "Category", "Item", "Qty", "Price Point Name", "Net Sales", "Transaction ID"]
data = data_Full[features]
data.head()

,Date,Time,Category,Item,Qty,Price Point Name,Net Sales,Transaction ID
0,12/31/16,16:52:04,1 Hot Drinks,Tea,1,20 oz,$3.75,d03AxLnaHYYYHNHlsJmK8xreV
1,12/31/16,16:49:47,1 Hot Drinks,Latte/Cappuccino,1,12 oz,$3.25,N0rqi3HeVh14y7WI7tv1EQmeV
2,12/31/16,16:44:51,1 Hot Drinks,Brew Coffee,4,12 oz,$7.40,9eGd92PVSyj1M1Csk8CPuSxeV
3,12/31/16,16:44:51,1 Hot Drinks,Espresso.double,1,Regular Price,$2.50,9eGd92PVSyj1M1Csk8CPuSxeV
4,12/31/16,16:44:51,1 Hot Drinks,Cappuccino,1,Regular Price,$3.35,9eGd92PVSyj1M1Csk8CPuSxeV


0) Location is static. Remove Time Zone.

1) SKU is not populated, so we will remove it.

2) Madifiers Applies does not seem applicable, we will remove it.

3) Look into discounts - looks very sparse. If we omit it, omit Gross Sales.

4) Tax is a function of Net Sales. Remove it.

%2.22 of purchases had a discount averaging $0.53 per item purchased with discount.

In [5]:
# Lets take a look at all of the categories per feature.
features_unique_count = {}
features_unique = {}
for feature in features:
    features_unique[feature] = data[feature].unique()
    print(feature + " has %d unique values." %(features_unique[feature].size))

Date has 362 unique values.
Time has 36016 unique values.
Category has 16 unique values.
Item has 138 unique values.
Qty has 30 unique values.
Price Point Name has 278 unique values.
Net Sales has 920 unique values.
Transaction ID has 100419 unique values.


In [6]:
print ("START TIME: " + str(time.asctime( time.localtime(time.time()))))
# Add some features that correspond to time
# Month
month = []
day_of_week = []
day_of_month = []
for i in range(len(data)):
    date = datetime.strptime((data["Date"][i]), '%m/%d/%y')
    month.append(date.month)
    day_of_month.append(date.day)
    
    if i % 50000 == 0:
        print i
# Day of the week
# Day of the month
alert(1)
print ("END TIME: " + str(time.asctime( time.localtime(time.time()))))

START TIME: Wed Apr 26 23:51:31 2017
0
50000
100000
150000
END TIME: Wed Apr 26 23:51:45 2017


In [7]:
# Add in weather data
data.insert(1,"Month",pd.Series(month))
data.insert(2,"day_of_month", pd.Series(day_of_month))
print data.head()
alert(1)

       Date  Month  day_of_month      Time      Category              Item  \
0  12/31/16     12            31  16:52:04  1 Hot Drinks               Tea   
1  12/31/16     12            31  16:49:47  1 Hot Drinks  Latte/Cappuccino   
2  12/31/16     12            31  16:44:51  1 Hot Drinks       Brew Coffee   
3  12/31/16     12            31  16:44:51  1 Hot Drinks   Espresso.double   
4  12/31/16     12            31  16:44:51  1 Hot Drinks        Cappuccino   

   Qty Price Point Name Net Sales             Transaction ID  
0    1            20 oz    $3.75   d03AxLnaHYYYHNHlsJmK8xreV  
1    1            12 oz    $3.25   N0rqi3HeVh14y7WI7tv1EQmeV  
2    4            12 oz    $7.40   9eGd92PVSyj1M1Csk8CPuSxeV  
3    1    Regular Price    $2.50   9eGd92PVSyj1M1Csk8CPuSxeV  
4    1    Regular Price    $3.35   9eGd92PVSyj1M1Csk8CPuSxeV  


In [ ]:
# Use the day of the week from a time before anything we would have data on as a reference and do - days % 7.